# IPFP/Sinkorn

In [ ]:
import numpy as np
import gurobipy as grb
import os
import pandas as pd
import time

In [ ]:
syntheticData = True
doGurobi = True
doIPFP1 = True
doIPFP1bis = True
doIPFP2 = True

In [ ]:
tol = 1E-9
maxite = 1000000
sigma = 0.1

In [ ]:
if syntheticData:
    seed = 777
    nbX = 100
    nbY = 100
    np.random.seed(seed)
    Xvals = np.random.uniform(0,1,nbX)
    Yvals = np.random.uniform(0,1,nbY)
    Phi = np.matmul(np.asmatrix(Xvals).T,np.asmatrix(Yvals))
    p = np.repeat(1/nbX, nbX)
    q = np.repeat(1/nbY, nbY)

In [ ]:
if not syntheticData:
    thePath = os.getcwd().split("veteran_students_mec_optim\\Pauline")[0]
    data = pd.read_csv(thePath + 'data_mec_optim\\marriage_personality-traits\\affinitymatrix.csv', sep=',')
    nbcar = 10
    AffMat = data.iloc[:nbcar, 1:nbcar + 1].values
    data = pd.read_csv(thePath + 'data_mec_optim\\marriage_personality-traits\\Xvals.csv', sep=',')
    Xvals = data.iloc[:, :nbcar].values
    data = pd.read_csv(thePath + 'data_mec_optim\\marriage_personality-traits\\Yvals.csv', sep=',')
    Yvals = data.iloc[:, :nbcar].values
    nobs = Xvals.shape[0]
    Xvals = (Xvals - np.mean(Xvals, axis=0)) / np.std(Xvals, axis=0, ddof=1)
    Yvals = (Yvals - np.mean(Yvals, axis=0)) / np.std(Yvals, axis=0, ddof=1)
    Phi = Xvals.dot(AffMat.dot(Yvals.T))
    p = np.repeat(1/nobs, nobs)
    q = np.repeat(1/nobs, nobs)
    nbX = len(p)
    nbY = len(q)

In [ ]:
nrow = min(8, nbX)
ncol = min(8, nbY)

In [ ]:
if doGurobi:
    coupleList = [(i, j) for i in range(nbY) for j in range(nbX)]
    ptm = time.time()
    m = grb.Model('marriage')
    couple = m.addVars(coupleList, obj=Phi.T.flatten().tolist(), name='couple')
    m.ModelSense = -1
    m.addConstrs((couple.sum('*', women) == p[women] for women in range(nbX)))
    m.addConstrs((couple.sum(men, '*') == q[men] for men in range(nbY)))
    m.optimize()
    diff = time.time() - ptm
    print('Time elapsed (Gurobi) = ', diff, 's.')
    if m.status == grb.GRB.Status.OPTIMAL:
        val_gurobi = m.objval
        pi = m.getAttr(grb.GRB.Attr.Pi)
        u_gurobi = pi[:nbX]
        v_gurobi = pi[nbX:nbX + nbY]
        print("Value of the problem (Gurobi) = ", val_gurobi)
        print(np.subtract(u_gurobi[:nrow], u_gurobi[nrow - 1]))
        print(np.add(v_gurobi[:ncol], u_gurobi[nrow - 1]))
        print('*************************')

In [ ]:
if doIPFP1:
    ptm = time.time()
    ite = 0

    K = np.exp(Phi/sigma)
    B = np.repeat(1, nbY).reshape(nbY,1)
    error = tol + 1
    
    while error > tol and ite < maxite:
        A = p/K.dot(B).flatten()
        KA = A.dot(K)
        error = np.max(abs(np.multiply(KA,B.flatten())-q))
        B = (q / KA).T
        ite = ite + 1
        
    u = - sigma * np.log(A)
    v = - sigma * np.log(B)
    pi =  np.exp(((Phi.T - u).T - np.tile(v, nbX).reshape(nbX, nbY)) / sigma)
    val = np.sum(np.multiply(pi,Phi)) - sigma * np.sum(np.multiply(pi,np.log(pi)))
    end = time.time() - ptm
    if ite >= maxite:
        print('Maximum number of iteations reached in IPFP1.')
    else:
        print('IPFP1 converged in ', ite, ' steps and ', end, 's.')
        print('Value of the problem (IPFP1) = ', val)
        print('Sum(pi*Phi) (IPFP1) = ', np.sum(np.multiply(pi,Phi)))
    print('*************************')

In [ ]:
if doIPFP1bis:
    ptm = time.time()
    ite = 0
    v = np.repeat(0, nbY)
    mu = - sigma * np.log(p)
    nu = - sigma * np.log(q)
    error = tol + 1
    while error > tol and ite < maxite:
        u = mu + sigma * np.log(np.sum(np.exp((Phi - np.tile(v, nbX).reshape(nbX, nbY))/sigma), axis=1)).flatten()
        KA = np.sum(np.exp((Phi.T - u).T / sigma), axis=0)
        error = np.max(np.abs(np.multiply(KA,np.exp(-v / sigma)) / q - 1))
        v = nu + sigma * np.log(KA)
        ite = ite + 1
    pi_bis = np.exp(((Phi.T - u).T - np.tile(v, nbX).reshape(nbX, nbY)) / sigma)
    val = np.sum(np.multiply(pi_bis,Phi)) - sigma * np.sum(np.multiply(pi_bis,np.log(pi_bis)))
    end = time.time() - ptm

    if ite >= maxite:
        print('Maximum number of iteations reached in IPFP1.')
    else:
        print('IPFP1bis converged in ', ite, ' steps and ', end, 's.')
        print('Value of the problem (IPFP1bis) = ', val)
        print('Sum(pi*Phi) (IPFP1bis) = ', np.sum(np.multiply(pi_bis,Phi)))
    print('*************************')

In [ ]:
if (doIPFP2):
    ptm = time.time()
    ite = 0
    v = np.repeat(0, nbY)
    mu = - sigma * np.log(p)
    nu = - sigma * np.log(q)
    error = tol + 1
    uprec = float('-inf')
    while error > tol and ite < maxite:
        vstar = np.max(Phi - v, axis=1)
        u = mu + vstar.flatten() + sigma * np.log(np.sum(np.exp((Phi - np.tile(v, nbX).reshape(nbX, nbY) - vstar)/sigma), axis=1)).flatten()
        error = np.max(np.abs(u - uprec))
        uprec = u
        ustar = np.max((Phi.T - u).T, axis=0)
        v = nu + ustar + sigma * np.log(np.sum(np.exp(((Phi.T - u).T - np.tile(ustar, nbX).reshape(nbX, nbY))/sigma), axis=0)).flatten()
        ite = ite + 1
    pi2 = np.exp(((Phi.T - u).T - np.tile(v, nbX).reshape(nbX, nbY)) / sigma)
    val = np.sum(np.multiply(pi2,Phi)) - sigma * np.sum(np.multiply(pi2,np.log(pi2)))
    end = time.time() - ptm
    if ite >= maxite:
        print('Maximum number of iteations reached in IPFP2.')
    else:
        print('IPFP2 converged in ', ite, ' steps and ', end, 's.')
        print('Value of the problem (IPFP2) = ', val)
        print('Sum(pi*Phi) (IPFP2) = ', np.sum(np.multiply(pi2,Phi)))
    print('*************************')

In [ ]:
pi

In [ ]:
pi_bis

In [ ]:
pi2